In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np

In [2]:
data = loadmat('trainfmri3d.mat')

/Users/alexiswu/Desktop/neurotech/lib/python3.10/site-packages/scipy/io/matlab/_mio.py:227: MatReadWarning: Duplicate variable name "None" in stream - replacing previous with new
Consider mio5.varmats_from_mat to split file into single variable files
  matfile_dict = MR.get_variables(variable_names)


In [48]:
mask = data['mask'].reshape(-1).astype(bool)
df_mask = pd.DataFrame(mask)

In [50]:
df_mask.to_csv('save_params/brain_mask.csv')

In [3]:
def load_and_mask(data):
    mask = data['mask'].reshape(-1).astype(bool)
    mask_region1 = data['mask_region1'].reshape(-1).astype(bool)
    mask_region2 = data['mask_region2'].reshape(-1).astype(bool)
    mask_region3 = data['mask_region3'].reshape(-1).astype(bool)
    mask_region4 = data['mask_region4'].reshape(-1).astype(bool)
    train_fmri3d = data['train_fmri3d'].reshape(-1,162).transpose()
    labels = pd.read_csv('train_labels.csv',header=None)
    whole_brain = []
    region1 = []
    region2 = []
    region3 = []
    region4 = []
    for d in train_fmri3d:
        whole_brain.append(np.ma.array(d, mask=~mask).filled(fill_value=0))
        region1.append(np.ma.array(d, mask=~mask_region1).filled(fill_value=0))
        region2.append(np.ma.array(d, mask=~mask_region2).filled(fill_value=0))
        region3.append(np.ma.array(d, mask=~mask_region3).filled(fill_value=0))
        region4.append(np.ma.array(d, mask=~mask_region4).filled(fill_value=0))
    regions = [region1, region2, region3, region4]
    return regions, labels, whole_brain

    

In [3]:
def load_and_mask(data):
    mask = data['mask'].reshape(-1).astype(bool)
    mask_region1 = data['mask_region1'].reshape(-1).astype(bool)
    mask_region2 = data['mask_region2'].reshape(-1).astype(bool)
    mask_region3 = data['mask_region3'].reshape(-1).astype(bool)
    mask_region4 = data['mask_region4'].reshape(-1).astype(bool)
    train_fmri3d = data['train_fmri3d'].reshape(-1,162).transpose()
    labels = pd.read_csv('train_labels.csv',header=None)
    whole_brain = []
    region1 = []
    region2 = []
    region3 = []
    region4 = []
    for d in train_fmri3d:
        whole_brain.append(np.ma.array(d, mask=~mask).filled(fill_value=0))
        region1.append(np.ma.array(d, mask=~mask_region1).filled(fill_value=0))
        region2.append(np.ma.array(d, mask=~mask_region2).filled(fill_value=0))
        region3.append(np.ma.array(d, mask=~mask_region3).filled(fill_value=0))
        region4.append(np.ma.array(d, mask=~mask_region4).filled(fill_value=0))
    regions = [region1, region2, region3, region4]
    return regions, labels, whole_brain

    

In [4]:
regions, labels, whole_brain = load_and_mask(data)


# Dimensionality Reduction
## PCA

In [124]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [128]:
def PCA_reduction(region):
    x = StandardScaler().fit_transform(region)
    pca = PCA(n_components=80)
    PCs = pca.fit_transform(x)
    return pca, pd.DataFrame(PCs)

In [ ]:
X_train, X_test = train_test_split(whole_brain, test_size=0.33, random_state=42)

In [129]:
pca,df_pca = PCA_reduction(whole_brain)

In [130]:
df_pca.head(5)

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,244.059561,-53.009625,-73.104909,2.226906,-9.682479,-7.241342,9.031807,-12.661569,36.587658,-3.113218,...,-0.314030,-0.939965,1.574927,1.818959,2.053352,1.051539,0.289606,-3.212223,-0.173186,0.975142
1,243.018123,-51.926004,-75.456479,-0.852447,-13.055425,-3.421077,7.007460,-7.827998,35.912729,2.243219,...,-3.664666,1.495062,-0.465855,-0.840735,2.236974,0.149188,-0.301855,0.192775,-0.240017,1.476314
2,241.482696,-50.156863,-76.809838,2.446407,-7.468902,-4.897464,7.272951,-5.987472,35.206925,0.943197,...,0.033392,2.569976,-1.248307,1.578977,0.174804,-3.457592,1.730467,3.893391,1.638536,1.231950
3,243.428585,-51.587863,-78.076047,4.392065,-5.883809,-5.533057,2.324079,-8.464100,31.614936,-4.744107,...,-1.429945,-0.096418,1.356137,-0.623611,-3.928607,-0.466383,-5.315796,-3.472207,-1.961481,-1.566169
4,241.409110,-51.942934,-76.968453,2.600461,-11.136642,-3.921068,7.639429,-3.220593,34.303432,-0.903921,...,-1.046664,0.062536,2.074404,0.573536,-0.701890,-0.243185,2.509017,0.621748,0.009618,0.920829


In [131]:
df_pca.to_csv('pca80_4regions.csv', index=False)

## Autoencoder

In [66]:
from keras.layers import Input, Dense, Dropout
from keras.models import Model

In [67]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [68]:
encoding_dim = 80
input = Input(shape=(163840,))
encoded = Dense(encoding_dim*4, activation='relu')(input)
encoded = Dropout(0.2)(encoded)
encoded = Dense(encoding_dim*2, activation='relu')(encoded)
encoded = Dense(encoding_dim, activation='relu')(encoded)


In [69]:
decoded = Dense(encoding_dim*2, activation='relu')(encoded)
decoded = Dense(encoding_dim*4, activation='relu')(decoded)
decoded = Dense(163840, activation='relu')(decoded)
autoencoder = Model(input, decoded)

In [70]:
autoencoder.compile(optimizer='adam', loss='MSE')

In [71]:
X_train, X_test = train_test_split(whole_brain, test_size=0.33, random_state=42)

In [72]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [73]:
autoencoder.fit(X_train, X_train,
                epochs=11,
                batch_size=64,
                shuffle=True,
                validation_data=(X_test, X_test))

Epoch 1/11
2/2 [==============================] - 3s 1s/step - loss: 0.2437 - val_loss: 0.2483
Epoch 2/11
2/2 [==============================] - 2s 899ms/step - loss: 0.2428 - val_loss: 0.2382
Epoch 3/11
2/2 [==============================] - 2s 922ms/step - loss: 0.2323 - val_loss: 0.2210
Epoch 4/11
2/2 [==============================] - 2s 903ms/step - loss: 0.2177 - val_loss: 0.2163
Epoch 5/11
2/2 [==============================] - 2s 912ms/step - loss: 0.2136 - val_loss: 0.2120
Epoch 6/11
2/2 [==============================] - 2s 896ms/step - loss: 0.2073 - val_loss: 0.2050
Epoch 7/11
2/2 [==============================] - 2s 888ms/step - loss: 0.2007 - val_loss: 0.2026
Epoch 8/11
2/2 [==============================] - 2s 883ms/step - loss: 0.1975 - val_loss: 0.1992
Epoch 9/11
2/2 [==============================] - 2s 912ms/step - loss: 0.1949 - val_loss: 0.1977
Epoch 10/11
2/2 [==============================] - 2s 912ms/step - loss: 0.1925 - val_loss: 0.1947
Epoch 11/11
2/2 [=====

In [74]:
encoder_middle = Model(input, encoded)

In [75]:
df_autoencoder = pd.DataFrame(encoder_middle.predict(np.stack(whole_brain)))

6/6 [==============================] - 0s 21ms/step


In [77]:
df_autoencoder.max().max()

2179.102

In [78]:
df_autoencoder.to_csv("autoencoder80_4regions.csv")

# Save Parameters

In [21]:
from joblib import dump



In [144]:
scaler_params = {
    'mean': list(scaler.mean_),
    'scale': list(scaler.scale_)
}
pca_params = {
    'components': pca.components_.tolist(),
    'explained_variance': pca.explained_variance_.tolist(),
    # Add any other necessary PCA attributes
}
params = {
    'scaler': scaler_params,
    'pca': pca_params
}

with open('save_params/pca80_params.json', 'w') as file:
    json.dump(params, file)

In [79]:
encoder_middle.save_weights('save_params/autoencoder_80_whole.h5')

In [80]:
# dump(pca, 'save_params/pca_80_whole.joblib')
dump(scaler, 'save_params/scaler_80_whole.joblib')

['save_params/scaler_80_whole.joblib']

In [81]:
from keras.models import Model

# Assume you have a Keras model defined as:
# model = Model(inputs=input, outputs=encoded)

# Serialize the model architecture to JSON
model_json = encoder_middle.to_json()

with open("save_params/autoencoder_80_all.json", "w") as json_file:
    json_file.write(model_json)